# Binding Functions

You can create your own functions to be called when a model is called as well. Outside of the functions that OpenAI creates. Let's create our own function to get the current time.

In [ ]:
pip install -q openai pytz

In [ ]:
import json
from datetime import datetime
import pytz

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_time(time_zone):
    """Get the current time in a given location"""
    time = datetime.now(pytz.timezone(time_zone))
    time_info = {
        "time_zone": time_zone,
        "time": str(time),
    }
    return json.dumps(time_info)

In [ ]:
# define a function
functions = [
    {
        "name": "get_current_time",
        "description": "Get the current time given a location",
        "parameters": {
            "type": "object",
            "properties": {
                "time_zone": {
                    "type": "string",
                    "description": "The time zone of the location we want to find the time for",
                },
            },
            "required": ["time_zone"],
        },
    }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the time in Minneapolis using a 24 hour clock?"
        #"content": "What's the size of the moon?"
    }
]

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [ ]:

print(response.choices[0].message.function_call)
#args = response.choices[0].message.function_call.arguments
if response.choices[0].message.function_call is None:
    args = {}
else:
    args = json.loads(response.choices[0].message.function_call.arguments)

    # Let's create another message that has the role of "function"
    messages.append(
        {
            "role": "function", #note this role is "function" this is how the model knows this the response is from the function
            "name": "get_current_time",
            "content": get_current_time(args["time_zone"]),
        }
    )


In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

print(json.dumps(messages, indent=2))
print (response.choices[0].message.content)